In [3]:
import pandas as pd

In [9]:
df = pd.read_excel('/Users/user/Documents/DF/리뷰분석프로젝트/review_example (1).xlsx')

In [11]:
df.columns = ['Product', 'Date', 'Buyer', 'ShelfLife', 'Packaging', 'Convenience', 'Review']

##텍스트 클렌징

In [18]:
import re

# 텍스트 정제 함수 정의
def text_clean(text):
    if pd.isnull(text):  # 값이 NaN일 경우 빈 문자열로 처리
        return ''

    # 텍스트 클리닝 진행
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'  # E-mail 제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL 제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'  # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'  # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s가-힣]'  # 한글 및 공백을 제외한 특수기호 제거
    text = re.sub(pattern, '', text)
    text = re.sub(r'\s+', ' ', text).strip()  # 중복된 공백 제거

    return text

# 'ReviewContent' 열을 리스트로 변환
reviews = df["Review"].astype(str).tolist()  # 모든 값을 문자열로 변환

# 텍스트 클리닝 적용
cleaned_reviews = list(map(text_clean, reviews))

# 클렌징된 리뷰 내용을 데이터프레임에 새로운 열로 추가
df['Review1'] = cleaned_reviews

<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\w'
<>:19: SyntaxWarning: invalid escape sequence '\w'
<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:11: SyntaxWarning: invalid escape sequence '\w'
<>:19: SyntaxWarning: invalid escape sequence '\w'
/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_42901/486446856.py:9: SyntaxWarning: invalid escape sequence '\.'
  pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'  # E-mail 제거
/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_42901/486446856.py:11: SyntaxWarning: invalid escape sequence '\w'
  pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'  # URL 제거
/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_42901/486446856.py:19: SyntaxWarning: invalid escape sequence '\w'
  pattern = '[^\w\s가-힣]'  # 한글 및 공백을 제외한 특수기호 제거


In [20]:
!pip install konlpy


In [21]:
from konlpy.tag import Okt
import re
import pandas as pd

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 텍스트 정제 및 ngram=1 생성 함수 정의 (단어 단위 형태소 추출)
def morphs_unigram(text):
    if pd.isnull(text):  # NaN일 경우 빈 문자열 처리
        return ''

    # 한글 전처리 (특수 문자 제거)
    text = re.sub(r'[^\w\s가-힣]', '', text)

    # 형태소 전체 추출 (ngram=1)
    tokens = okt.morphs(text)

    return " ".join(tokens)

# 텍스트 정제 및 ngram=2 생성 함수 정의
def morphs_bigram(text):
    if pd.isnull(text):  # NaN일 경우 빈 문자열 처리
        return ''

    # 한글 전처리 (특수 문자 제거)
    text = re.sub(r'[^\w\s가-힣]', '', text)

    # 형태소 전체 추출
    tokens = okt.morphs(text)

    # 2-gram 생성
    bigrams = [" ".join(tokens[i:i+2]) for i in range(len(tokens)-1)]

    return " ".join(bigrams)

# 명사만 추출하는 함수 정의
def extract_nouns(text):
    if pd.isnull(text):  # NaN일 경우 빈 문자열 처리
        return ''

    # 한글 전처리 (특수 문자 제거)
    text = re.sub(r'[^\w\s가-힣]', '', text)

    # 명사 추출
    nouns = okt.nouns(text)

    return " ".join(nouns)

# 'Review1' 열에 대해 형태소 분석 적용 (ngram=1, ngram=2, 명사 추출)
df['new1'] = df['Review1'].astype(str).apply(morphs_unigram)  # ngram=1 저장
df['new2'] = df['Review1'].astype(str).apply(morphs_bigram)   # ngram=2 저장
df['N'] = df['Review1'].astype(str).apply(extract_nouns)      # 명사 추출 저장

# 결과 확인
df[['Review1', 'new1', 'new2', 'N']].head()


,Review1,new1,new2,N
0,속편한 고려은단 비타민 중성비타민 정을 구매하여 복용해 본 후 매우 만족스러운 경험...,속편 한 고려 은단 비타민 중성 비타민 정 을 구매 하여 복용 해 본 후 매우 만족...,속편 한 한 고려 고려 은단 은단 비타민 비타민 중성 중성 비타민 비타민 정 정 을...,속편 고려 은단 비타민 중성 비타민 정 구매 복용 후 매우 경험 비타민 우리 몸 역...
1,만성 위염 식도염으로 고생하는 사람이예요 속편한 비타민이 있는 줄 몰랐어요 이제부터...,만성 위염 식도염 으로 고생 하는 사람 이 예요 속편 한 비타민 이 있는 줄 몰랐어...,만성 위염 위염 식도염 식도염 으로 으로 고생 고생 하는 하는 사람 사람 이 이 예...,만성 위염 식도염 고생 사람 속편 비타민 줄 이제 때 피부 하니 이제 속 수 사진 ...
2,알이 커서 걱정이 많았는데 먹는데는 아무 지장이 없습니다 처음에는 가루로 메가도스를...,알 이 커서 걱정 이 많았는데 먹는데는 아무 지장 이 없습니다 처음 에는 가루 로 ...,알 이 이 커서 커서 걱정 걱정 이 이 많았는데 많았는데 먹는데는 먹는데는 아무 아...,알 커서 걱정 아무 지장 처음 가루 메 도스 가끔 현상 걸 일단 기분 식후 알 소화...
3,양도 많고 속도 편한 비타민라 부담없이 먹기에 좋아요 믿고 먹는 고려은단 비타민입니...,양도 많고 속도 편한 비타민 라 부담 없이 먹기 에 좋아요 믿고 먹는 고려 은단 비...,양도 많고 많고 속도 속도 편한 편한 비타민 비타민 라 라 부담 부담 없이 없이 먹...,양도 속도 비타민 부담 먹기 고려 은단 비타민 적극 추천 이미지 보기
4,고러은단 비타민제품은 종류별로 여러번복용해 보았답니다 이번제품은 약크기는 조금크지만...,고러은 단 비타민 제품 은 종류 별로 여러 번복 용해 보았답니다 이번 제품 은 약 ...,고러은 단 단 비타민 비타민 제품 제품 은 은 종류 종류 별로 별로 여러 여러 번복...,단 비타민 제품 종류 별로 번복 용해 이번 제품 크기 조금 알 함량 중성 제품 맘 ...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19927 entries, 0 to 19926
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Product      19927 non-null  object
 1   Date         19927 non-null  object
 2   Buyer        6133 non-null   object
 3   ShelfLife    19927 non-null  object
 4   Packaging    19927 non-null  object
 5   Convenience  19927 non-null  object
 6   Review       19927 non-null  object
 7   Review1      19927 non-null  object
 8   new1         19927 non-null  object
 9   new2         19927 non-null  object
 10  N            19927 non-null  object
dtypes: object(11)
memory usage: 1.7+ MB


## 불용어 처리

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# 불용어 리스트
stopwords = [
    '가량', '가지', '각', '간', '갖은', '개', '개국', '개년', '개소', '개월', '걔', '거', '거기', '거리', '건', '것', '겨를',
    '격', '겸', '고', '군', '군데', '권', '그', '그거', '그것', '그곳', '그까짓', '그네', '그녀', '그놈', '그대', '그래', '그래도',
    '그서', '그러나', '그러니', '그러니까', '그러다가', '그러면', '그러면서', '그러므로', '그러자', '그런', '그런데', '그럼', '그렇지만', '그루',
    '그리고', '그리하여', '그분', '그이', '그쪽', '근', '근데', '글쎄', '글쎄요', '기', '김', '나', '나름', '나위', '남짓', '내', '냥',
    '너', '너희', '네', '네놈', '녀석', '년', '년대', '년도', '놈', '누구', '니', '다른', '다만', '단', '달', '달러', '당신', '대', '대로',
    '더구나', '더욱이', '데', '도', '동', '되', '두', '두세', '두어', '둥', '듯', '듯이', '등', '등등', '등지', '따라서', '따름', '따위', '딴',
    '때문', '또', '또는', '또한', '리', '마당', '마련', '마리', '만', '만큼', '말', '매', '맨', '명', '몇', '몇몇', '모', '모금', '모든', '무렵',
    '무슨', '무엇', '뭐', '뭣', '미터', '및', '바', '바람', '바퀴', '박', '발', '발짝', '번', '벌', '법', '별', '본', '부', '분', '뻔', '뿐',
    '살', '새', '서너', '석', '설', '섬', '세', '세기', '셈', '쇤네', '수', '순', '스무', '승', '시', '시간', '식', '씨', '아', '아냐', '아니',
    '아니야', '아무', '아무개', '아무런', '아아', '아이', '아이고', '아이구', '야', '약', '양', '얘', '어', '어느', '어디', '어머', '언제', '에이',
    '엔', '여기', '여느', '여러', '여러분', '여보', '여보세요', '여지', '역시', '예', '옛', '오', '오랜', '오히려', '온', '온갖', '올', '왜냐하면',
    '왠', '외', '요', '우리', '원', '월', '웬', '위', '음', '응', '이', '이거', '이것', '이곳', '이놈', '이래', '이런', '이런저런', '이른바',
    '이리하여', '이쪽', '일', '일대', '임마', '자', '자기', '자네', '장', '저', '저것', '저기', '저놈', '저런', '저쪽', '저편', '저희', '적',
    '전', '점', '제', '조', '주', '주년', '주일', '줄', '중', '즈음', '즉', '지', '지경', '지난', '집', '짝', '쪽', '쯤', '차', '참', '채',
    '척', '첫', '체', '초', '총', '측', '치', '큰', '킬로미터', '타', '터', '턱', '톤', '통', '투', '판', '퍼센트', '편', '평', '푼', '하기야',
    '하긴', '하물며', '하지만', '한', '한두', '한편', '허허', '헌', '현', '호', '혹은', '회', '흥',
    # 추가 불용어 (조사, 접속사 등)
    '은', '는', '이', '가', '을', '를', '와', '과', '도', '에', '의', '년', '월', '일', '시', '분','이미지','펼쳐','보기'
]

# 불용어 처리 함수 정의
def remove_stopwords(text):
    # 각 문장을 띄어쓰기 단위로 쪼개어 불용어를 제거
    return " ".join([word for word in text.split() if word not in stopwords])

# 'new1', 'new2', 'N'에 대해 불용어 제거 적용
df['new1_cleaned'] = df['new1'].astype(str).apply(remove_stopwords)
df['new2_cleaned'] = df['new2'].astype(str).apply(remove_stopwords)
df['N_cleaned'] = df['N'].astype(str).apply(remove_stopwords)



## 벡터화

In [28]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 적용 함수 정의
def get_top_n_words(vectorizer, X, n=10, filter_repeated_bigrams=False):
    # 단어 목록과 빈도를 합쳐서 데이터프레임 생성
    word_counts = pd.DataFrame({
        'word': vectorizer.get_feature_names_out(),
        'count': X.toarray().sum(axis=0)
    })

    # filter_repeated_bigrams가 True인 경우에만 중복된 단어로 구성된 bigram 필터링
    if filter_repeated_bigrams:
        word_counts = word_counts[~word_counts['word'].apply(lambda x: len(set(x.split())) == 1)]

    # 빈도순으로 정렬
    word_counts = word_counts.sort_values(by='count', ascending=False)

    # 상위 n개의 단어 반환
    return word_counts.head(n)

# CountVectorizer 적용 (new1과 N은 unigram, new2는 bigram)
vectorizer_new1 = CountVectorizer()
vectorizer_new2 = CountVectorizer(ngram_range=(2, 2))  # n-gram 2 설정
vectorizer_N = CountVectorizer()

# 각 cleaned 열에 대해 CountVectorizer 적용
X_new1 = vectorizer_new1.fit_transform(df['new1_cleaned'])
X_new2 = vectorizer_new2.fit_transform(df['new2_cleaned'])  # 2-gram 적용
X_N = vectorizer_N.fit_transform(df['N_cleaned'])

# 중요한 단어 상위 10개 추출 (new2에서만 중복 bigram 필터링)
top_new1_words = get_top_n_words(vectorizer_new1, X_new1, n=30)
top_new2_words = get_top_n_words(vectorizer_new2, X_new2, n=30, filter_repeated_bigrams=True)
top_N_words = get_top_n_words(vectorizer_N, X_N, n=30)




## count 기반 중요단어 출력

In [31]:
# 결과 출력
print("new1에서 중요한 단어들:")
print(top_new1_words)


new1에서 중요한 단어들:
        word  count
4729     비타민   5999
8277     좋아요   4597
7865     재구매   4137
784       구매   3796
4127      배송   3573
3241      먹고   2177
4989      사용   2045
8090      제품   2006
5317      속이   1798
3314      먹기   1770
8941      처음   1769
583       고려   1647
2843      리뷰   1534
9531      포장   1488
7730      자의   1488
7491     입니다   1471
1199     꾸준히   1411
7207      은단   1399
7006      용기   1386
8474      주문   1381
4752     빠르고   1340
10006     항상   1332
10215    했어요   1233
3435      먹어   1215
10097     해서   1144
8554      중성   1144
4362      복용   1144
3329      먹는   1138
7194      으로   1061
198    감사합니다    956


In [33]:
print("\nnew2에서 중요한 단어들:")
print(top_new2_words)


new2에서 중요한 단어들:
           word  count
53697     처음 리뷰   1488
4138      구매 자의   1488
46422     자의 처음   1488
3117      고려 은단   1388
21939    배송 빠르고   1216
52119    중성 비타민    881
27067     빠른 배송    695
15618    먹고 있어요    680
40481    용기 재구매    588
42019    은단 비타민    519
28501    사진 비디오    515
15613   먹고 있습니다    490
4280     구매 했어요    483
16398    먹기 편하고    397
49082    제품 입니다    371
30835    속편 비타민    370
51856    주문 했어요    330
21782  배송 감사합니다    323
26587    빈속 먹어도    323
16622     먹는 제품    320
16575    먹는 비타민    315
25454    비타민 고려    312
47226    재구매 배송    300
26892   빠르고 좋아요    279
47559    재구매 항상    275
6547     꾸준히 먹어    274
50555    좋아요 구매    274
20562     믿고 먹는    259
26171    비타민 이라    258
6519     꾸준히 먹고    257


In [35]:
print("\nN에서 중요한 단어들:")
print(top_N_words)


N에서 중요한 단어들:
     word  count
1317  비타민   5999
290    구매   3796
1132   배송   3573
1378   사용   2045
2504   제품   2006
940    먹기   1770
2738   처음   1769
187    고려   1647
827    리뷰   1534
2341   자의   1488
2984   포장   1488
2145   은단   1399
2047   용기   1386
2576   주문   1381
3104   항상   1332
1217   복용   1144
2607   중성   1144
1541   속편    952
1326   빈속    658
2      가격    634
3201   효과    624
171    계속    544
1387   사진    534
300    구입    527
1298  비디오    515
1475   선물    468
1237   부담    465
227    공복    439
1741   아주    435
1534   속도    429


## 감성분석

In [37]:
# SentiWord_Dict.txt 파일을 로드하는 코드


# 파일 읽기
with open('SentiWord_Dict.txt', 'r', encoding='utf-8') as f:
    senti_dict = {}
    for line in f:
        # 각 줄의 단어와 감성 점수를 구분
        parts = line.strip().split('\t')

        # 유효한 단어와 감성 점수가 있는 경우에만 처리
        if len(parts) == 2:
            word, score = parts
            senti_dict[word] = float(score)

# 감성 사전의 일부를 확인
list(senti_dict.items())[:50]  # 상위 10개 단어와 감성 점수를 확인


FileNotFoundError: [Errno 2] No such file or directory: 'SentiWord_Dict.txt'

In [39]:
len(senti_dict)

NameError: name 'senti_dict' is not defined

In [ ]:
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 'Review' 열에 대해 토큰화 적용
def tokenize_text(text):
    if pd.isnull(text):  # NaN 값 처리
        return []
    return okt.morphs(text)  # 형태소 분석을 통해 단어 단위로 토큰화

# 'Review' 열에 대해 토큰화 적용하여 새로운 열 'tokenized_review'에 저장
df['tokenized_review'] = df['Review'].apply(tokenize_text)

# 결과 확인
df[['Review', 'tokenized_review']].head()


,Review,tokenized_review
0,"속편한 고려은단 비타민C 1000 중성비타민C 120정을 구매하여 복용해 본 후, ...","[속편, 한, 고려, 은단, 비타민, C, 1000, 중성, 비타민, C, 120,..."
1,만성 위염 식도염으로 고생하는 사람이예요\n속편한 비타민이 있는 줄 몰랐어요\n이제...,"[만성, 위염, 식도염, 으로, 고생, 하는, 사람, 이, 예요, \n, 속편, 한..."
2,알이 커서 걱정이 많았는데 먹는데는 아무 지장이 없습니다. 처음에는 가루로 메가도스...,"[알, 이, 커서, 걱정, 이, 많았는데, 먹는데는, 아무, 지장, 이, 없습니다,..."
3,양도 많고 속도 편한 비타민C라 부담없이 먹기에 좋아요.\n믿고 먹는 고려은단 비타...,"[양도, 많고, 속도, 편한, 비타민, C, 라, 부담, 없이, 먹기, 에, 좋아요..."
4,고러은단 비타민제품은 종류별로 여러번복용해 보았답니다. 이번제품은 약크기는 조금크지...,"[고러은, 단, 비타민, 제품, 은, 종류, 별로, 여러, 번복, 용해, 보았답니다..."


In [ ]:
# 3. 감성 점수 조회 및 4. 감성 점수 합산
def sentiment_analysis(tokenized_review):
    sentiment_score = 0  # 초기 감성 점수
    for word in tokenized_review:
        # 감성 사전에서 단어를 찾아 감성 점수를 더함 (사전에 없으면 0으로 처리)
        sentiment_score += senti_dict.get(word, 0)
    return sentiment_score

# 5. 감성 분석 결과 해석 (긍정/부정)
def classify_sentiment(score):
    if score > 0:
        return "긍정"
    elif score < 0:
        return "부정"
    else:
        return "중립"

# 토큰화된 리뷰에서 감성 점수를 계산하여 새로운 열 추가
df['sentiment_score'] = df['tokenized_review'].apply(sentiment_analysis)

# 감성 점수에 따른 긍정/부정 판단을 위한 열 추가
df['sentiment_label'] = df['sentiment_score'].apply(classify_sentiment)

# 결과 확인
df[['Review', 'tokenized_review', 'sentiment_score', 'sentiment_label']].head()


,Review,tokenized_review,sentiment_score,sentiment_label
0,"속편한 고려은단 비타민C 1000 중성비타민C 120정을 구매하여 복용해 본 후, ...","[속편, 한, 고려, 은단, 비타민, C, 1000, 중성, 비타민, C, 120,...",19.0,긍정
1,만성 위염 식도염으로 고생하는 사람이예요\n속편한 비타민이 있는 줄 몰랐어요\n이제...,"[만성, 위염, 식도염, 으로, 고생, 하는, 사람, 이, 예요, \n, 속편, 한...",3.0,긍정
2,알이 커서 걱정이 많았는데 먹는데는 아무 지장이 없습니다. 처음에는 가루로 메가도스...,"[알, 이, 커서, 걱정, 이, 많았는데, 먹는데는, 아무, 지장, 이, 없습니다,...",-4.0,부정
3,양도 많고 속도 편한 비타민C라 부담없이 먹기에 좋아요.\n믿고 먹는 고려은단 비타...,"[양도, 많고, 속도, 편한, 비타민, C, 라, 부담, 없이, 먹기, 에, 좋아요...",2.0,긍정
4,고러은단 비타민제품은 종류별로 여러번복용해 보았답니다. 이번제품은 약크기는 조금크지...,"[고러은, 단, 비타민, 제품, 은, 종류, 별로, 여러, 번복, 용해, 보았답니다...",2.0,긍정


In [ ]:
df_sentiment=df[['Review','sentiment_score','sentiment_label']]

In [ ]:
df_sentiment

,Review,sentiment_score,sentiment_label
0,"속편한 고려은단 비타민C 1000 중성비타민C 120정을 구매하여 복용해 본 후, ...",19.0,긍정
1,만성 위염 식도염으로 고생하는 사람이예요\n속편한 비타민이 있는 줄 몰랐어요\n이제...,3.0,긍정
2,알이 커서 걱정이 많았는데 먹는데는 아무 지장이 없습니다. 처음에는 가루로 메가도스...,-4.0,부정
3,양도 많고 속도 편한 비타민C라 부담없이 먹기에 좋아요.\n믿고 먹는 고려은단 비타...,2.0,긍정
4,고러은단 비타민제품은 종류별로 여러번복용해 보았답니다. 이번제품은 약크기는 조금크지...,2.0,긍정
...,...,...,...
19922,한달사용기먹기편하고 좋아요!!\n이미지 펼쳐보기\n0\n이 구매자의 처음 리뷰보기,2.0,긍정
19923,혓바늘이 돋고 너무 피곤해서 시켰는데 먹으니 바로 염증 사라지고 쌩쌩해지는 기분\n...,-2.0,부정
19924,위장속쓰림 때문에 꺼렸는데 고려은단꺼는 괜찮네요\n이미지 펼쳐보기\n0,0.0,중립
19925,딸에게선물했는데 잘복용중입니다\n0,1.0,긍정


## 토픽 모델링

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# LDA 모델 정의 (토픽 개수는 5로 설정)
num_topics = 5
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)

# LDA 모델 피팅
lda.fit(X_new1)

# 각 토픽별 핵심 단어 추출 함수
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

# 단어 목록 (feature names) 가져오기
feature_names = vectorizer_new1.get_feature_names_out()

# 각 토픽별로 상위 10개의 핵심 단어를 출력
no_top_words = 10
display_topics(lda, feature_names, no_top_words)


Topic 1:
좋아요 먹기 편하고 먹어도 재구매 사용 속이 비타민 빈속 부담
Topic 2:
비타민 속이 먹어 중성 먹고 으로 구매 속편 꾸준히 복용
Topic 3:
배송 포장 빠르고 좋아요 재구매 감사합니다 빠른 좋습니다 만족합니다 받았습니다
Topic 4:
주문 했어요 비타민 구매 해서 재구매 선물 구입 효과 했습니다
Topic 5:
구매 재구매 처음 리뷰 자의 비타민 입니다 제품 사용 항상


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer로 벡터화
vectorizer_new1_tfid = TfidfVectorizer()
X_new_tfid = vectorizer_new1_tfid.fit_transform(df['new1_cleaned'])

In [ ]:
# LDA 모델 피팅
lda.fit(X_new_tfid)

# 단어 목록 (feature names) 가져오기
feature_names = vectorizer_new1_tfid.get_feature_names_out()

# 각 토픽별로 상위 10개의 핵심 단어를 출력
no_top_words = 10
display_topics(lda, feature_names, no_top_words)

Topic 1:
좋아요 먹어도 비타민 빈속 먹기 사용 부담 속이 쓰림 중성
Topic 2:
먹고 비타민 있어요 있습니다 속이 재구매 꾸준히 먹어 구매 해서
Topic 3:
배송 좋아요 빠르고 포장 먹기 좋습니다 재구매 편하고 빨라요 편해요
Topic 4:
했어요 주문 비타민 구매 저렴하게 먹어 먹던거라 재구매 샀어요 했습니다
Topic 5:
재구매 감사합니다 구매 입니다 먹는 비타민 고려 항상 제품 빠른


In [ ]:
from sklearn.cluster import KMeans

# 1. KMeans 클러스터링 모델 정의 (클러스터 개수는 5로 설정)
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# 2. KMeans 모델 피팅 및 예측
cluster_labels = kmeans.fit_predict(X_new1) # count vectorize 기반으로 함 tfid는 영 별로

# 3. 클러스터 레이블을 데이터프레임에 추가
df['cluster_label'] = cluster_labels

# 결과 확인
df[['new1_cleaned', 'cluster_label']].head()


,new1_cleaned,cluster_label
0,속편 고려 은단 비타민 중성 비타민 정 구매 하여 복용 해 후 매우 만족스러운 경험...,3
1,만성 위염 식도염 으로 고생 하는 사람 예요 속편 비타민 있는 몰랐어요 이제 부터 ...,3
2,알 커서 걱정 많았는데 먹는데는 지장 없습니다 처음 에는 가루 로 메 도스 했는데 ...,2
3,양도 많고 속도 편한 비타민 라 부담 없이 먹기 좋아요 믿고 먹는 고려 은단 비타민...,3
4,고러은 비타민 제품 종류 별로 번복 용해 보았답니다 이번 제품 크기 조금 크지만 알...,3


In [ ]:
df[df['cluster_label'] == 0]['Review']

,Review
15,한달사용기포장이 꼼꼼해서 좋아요 먹기 편하고 아침에 먹어도 진짜로 속이 안쓰려요\n...
41,한달사용기생각보다는 알약 크기가 조금 크기는 한데요~\n잘 챙겨서 먹고 있어요~\n...
46,"한달사용기비타민C를 꼭 복용해야된다고해서지인소개로 사게된제품인데,다좋은데,알맹이가 ..."
58,몇번째 구매인지 기억 하기 어려울 정도로 구매했습니다 ^^~ 사무실에 두고 피곤 할...
66,한달사용기재구매비타민c는 고려 제품만 먹습니다\n보이는 효과는 없지만 꾸준히 먹다보...
...,...
19841,한달사용기포장이 전에 비해 조아진것 같아요\n맘에 듭니다\n0\n이 구매자의 처음 ...
19843,한달사용기속쓰림없고 먹기 편리해서 재구매합니다\n0\n이 구매자의 처음 리뷰보기
19884,한달사용기아주 잘쓰고이ㅛ어요!!\n0\n이 구매자의 처음 리뷰보기
19903,한달사용기비타민먹고나면 나는 특유의 속쓰림같은것도 없고 되게좋았어요 꾸준히복용 예정...


In [ ]:
df[df['cluster_label'] == 1]['Review']

,Review
13,속편한 버전인거를 모르고 샀네요ㅠㅠ\n신맛이없긴한데 맛은 그닥이고 빈속에도 먹을 수...
14,재구매토요일주문 일요일에 받았어요 비타민 똑 떨어졌었는데 배송 빨라 좋아요! :)소...
19,빈속에 먹어도되어서 꾸준히 구매중입니다.\n공복으로 운동할 때도 먹고 갈수있어서 좋...
23,늘 먹어오던 비타민이에요 속이 편해서 좋아요. 배송 빨르고.포장도 꼼꼼해요 추천합니...
25,재구매엄마가 꾸준히 드시는 비타민이에요. 배송이 정말 빨라서 좋아요\n이미지 펼쳐보...
...,...
19869,비타민중에서 제일 나은 것 같아요 배송도 빠르고 좋아요\n0
19873,배송도 빠르고 제품 좋아요\n0
19878,좋아여좋앙져좋아여좋아요\n이미지 펼쳐보기\n0
19887,배송이 빨라서 좋아요~~\n0


In [ ]:
df[df['cluster_label'] == 2]['Review']

,Review
2,알이 커서 걱정이 많았는데 먹는데는 아무 지장이 없습니다. 처음에는 가루로 메가도스...
5,.\n재구매 100프로\n사진/비디오 수\n4\n이미지 펼쳐보기\n0
6,배송이 너무 빨라서 놀랐어요 구매 당일에 온 것 같은데요??!! 먹어봤는데 정말 속...
12,"재구매꾸준히 먹는 제품으로 먹기 편하구요, 컨디션에도 도움되는거 같아요~ 그리고 제..."
20,그동안 해외직구로 저렴한 영양제 먹다가 고려은단으로 싹 다 바꾸는 중입니다. 좀 더...
...,...
19921,항상 먹는제품입니다.\n0
19923,혓바늘이 돋고 너무 피곤해서 시켰는데 먹으니 바로 염증 사라지고 쌩쌩해지는 기분\n...
19924,위장속쓰림 때문에 꺼렸는데 고려은단꺼는 괜찮네요\n이미지 펼쳐보기\n0
19925,딸에게선물했는데 잘복용중입니다\n0


In [ ]:
df[df['cluster_label'] == 3]['Review']

,Review
0,"속편한 고려은단 비타민C 1000 중성비타민C 120정을 구매하여 복용해 본 후, ..."
1,만성 위염 식도염으로 고생하는 사람이예요\n속편한 비타민이 있는 줄 몰랐어요\n이제...
3,양도 많고 속도 편한 비타민C라 부담없이 먹기에 좋아요.\n믿고 먹는 고려은단 비타...
4,고러은단 비타민제품은 종류별로 여러번복용해 보았답니다. 이번제품은 약크기는 조금크지...
7,종합영양제를 먹다가 잘안먹게되서 단품으로 필요한 영양제만 먹으려고 해요 그중에 하나...
...,...
19913,비타민 씨는 위장장애로\n잘못먹는데' .이건 속편하다고해서\n사봤어요.\n괜찮길바랍...
19916,비타민c먹으면 속쓰리다고 하셔서 주문해봤어요! 효과좋으면 좋겠네요.\n이미지 펼쳐보...
19917,"비타민C 속쓰려서 항상 식후 바로 먹었는데, 이제품은 공복에 먹어도 되니까 편할거같..."
19918,속편한 고려은단 비타민C1000\n0


In [ ]:
# 각 군집별 핵심 단어 추출 함수 (KMeans의 cluster_centers_ 기반)
def extract_cluster_keywords_kmeans(vectorizer, kmeans_model, top_n=10):
    feature_names = vectorizer.get_feature_names_out()  # 단어 목록
    for cluster_num, centroid in enumerate(kmeans_model.cluster_centers_):
        # 클러스터 중심점에서 중요한 단어의 순위를 정렬
        sorted_indices = centroid.argsort()[::-1]  # 높은 값 순서대로 정렬

        # 상위 n개의 단어 추출
        top_words = [feature_names[i] for i in sorted_indices[:top_n]]
        top_words_str = ", ".join(top_words)
        print(f"군집 {cluster_num}의 핵심 단어: {top_words_str}\n")

# 군집의 개수 설정 (KMeans에서 사용한 것과 동일하게)
num_clusters = 5
# 군집별 핵심 단어 추출
extract_cluster_keywords_kmeans(vectorizer_new1, kmeans, top_n=15)


군집 0의 핵심 단어: 구매, 처음, 리뷰, 자의, 사용, 용기, 좋아요, 재구매, 비타민, 먹고, 속이, 꾸준히, 있어요, 입니다, 먹기

군집 1의 핵심 단어: 좋아요, 재구매, 배송, 빠르고, 먹기, 비타민, 포장, 제품, 속이, 너무, 사용, 편하고, 먹고, 용기, 구매

군집 2의 핵심 단어: 재구매, 배송, 제품, 구매, 먹고, 먹기, 포장, 꾸준히, 감사합니다, 입니다, 빠르고, 주문, 항상, 속이, 좋습니다

군집 3의 핵심 단어: 비타민, 고려, 은단, 중성, 구매, 속이, 재구매, 먹고, 속편, 했어요, 으로, 제품, 주문, 먹는, 먹어



In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# 각 군집별 상위 10개의 TF-IDF 점수가 높은 단어를 추출하고 출력하는 함수
def print_top_tfidf_words_per_cluster(vectorizer, X, cluster_labels, num_clusters, top_n=10):
    feature_names = np.array(vectorizer.get_feature_names_out())  # 단어 목록
    for cluster_num in range(num_clusters):
        # 해당 클러스터에 속한 문서들만 선택
        cluster_indices = np.where(cluster_labels == cluster_num)  # 특정 클러스터 인덱스 선택
        X_cluster = X[cluster_indices]

        # 만약 클러스터에 문서가 하나만 있는 경우, 평균 계산이 아니라 바로 사용
        if isinstance(X_cluster, csr_matrix):
            X_cluster = X_cluster.toarray()  # 희소 행렬을 배열로 변환

        if X_cluster.shape[0] > 1:
            tfidf_means = np.mean(X_cluster, axis=0).flatten()  # 1D로 변환하여 사용
        else:
            tfidf_means = X_cluster.flatten()  # 하나의 문서일 경우 flatten만 수행

        # TF-IDF 점수가 높은 순서대로 정렬
        sorted_indices = np.argsort(tfidf_means)[::-1]  # 높은 순서대로 정렬

        # 상위 n개의 단어 추출
        top_words = feature_names[sorted_indices[:top_n]]
        top_scores = tfidf_means[sorted_indices[:top_n]]

        # 결과 출력
        print(f"\n군집 {cluster_num}의 상위 {top_n}개의 TF-IDF 점수 높은 단어:")
        for word, score in zip(top_words, top_scores):
            print(f"{word}: {score:.4f}")

# 군집의 개수 설정 (KMeans에서 사용한 것과 동일하게)
num_clusters = 5

# 군집별 상위 10개의 TF-IDF 점수가 높은 단어 출력
print_top_tfidf_words_per_cluster(vectorizer_new1, X_new_tfid, cluster_labels, num_clusters, top_n=10)



군집 0의 상위 10개의 TF-IDF 점수 높은 단어:
자의: 0.2436
리뷰: 0.2422
처음: 0.2340
구매: 0.1984
사용: 0.1231
용기: 0.1126
좋아요: 0.0459
재구매: 0.0424
먹고: 0.0401
비타민: 0.0277

군집 1의 상위 10개의 TF-IDF 점수 높은 단어:
좋아요: 0.2986
배송: 0.0693
빠르고: 0.0658
재구매: 0.0605
먹기: 0.0387
편하고: 0.0258
제품: 0.0233
비타민: 0.0232
너무: 0.0227
속이: 0.0199

군집 2의 상위 10개의 TF-IDF 점수 높은 단어:
배송: 0.0528
재구매: 0.0509
감사합니다: 0.0324
제품: 0.0284
먹고: 0.0282
빠르고: 0.0247
빠른: 0.0240
좋습니다: 0.0237
항상: 0.0221
먹기: 0.0213

군집 3의 상위 10개의 TF-IDF 점수 높은 단어:
비타민: 0.1751
고려: 0.0662
은단: 0.0577
중성: 0.0412
먹는: 0.0324
속이: 0.0298
구매: 0.0286
속편: 0.0282
재구매: 0.0263
입니다: 0.0249

군집 4의 상위 10개의 TF-IDF 점수 높은 단어:
